In [1]:
import pandas as pd
from pandasql import sqldf
q = lambda x: sqldf(x, globals())

In [11]:
activity = pd.DataFrame({'player_id' : [1, 1, 2, 3, 3], 
                         'device_id' : [2, 2, 3, 1, 4],
                         'event_date' : ['2016-03-01', '2016-03-02', 
                                         '2017-06-25', '2016-03-02', '2018-07-03'],
                         'games_played' : [5, 6, 1, 0, 5]})
activity['event_date']=pd.to_datetime(activity['event_date']).dt.date

In [3]:
activity

,player_id,device_id,event_date,games_played
0,1,2,2016-03-01,5
1,1,2,2016-05-02,6
2,2,3,2017-06-25,1
3,3,1,2016-03-02,0
4,3,4,2018-07-03,5


### Write an SQL query that reports the first login date for each player.

In [4]:
main_table = q('''SELECT player_id, event_date,
                  rank() OVER (PARTITION BY player_id ORDER BY event_date) as rank_date 
                  FROM activity''')

In [5]:
main_table

,player_id,event_date,rank_date
0,1,2016-03-01,1
1,1,2016-05-02,2
2,2,2017-06-25,1
3,3,2016-03-02,1
4,3,2018-07-03,2


In [6]:
q('''SELECT player_id, event_date
     FROM main_table
     WHERE rank_date = 1''')

,player_id,event_date
0,1,2016-03-01
1,2,2017-06-25
2,3,2016-03-02


### Write a SQL query that reports the device that is first logged in for each player.

In [7]:
q('''SELECT player_id, device_id
     FROM (SELECT player_id, device_id,
           rank() OVER (PARTITION BY player_id ORDER BY event_date) as rank_date 
           FROM activity)
     WHERE rank_date = 1''')

,player_id,device_id
0,1,2
1,2,3
2,3,1


### Write an SQL query that reports for each player and date, how many games played so far by the player. That is, the total number of games played by the player until that date.

In [8]:
q('''SELECT x.player_id, x.event_date
     FROM activity x''')

,player_id,event_date
0,1,2016-03-01
1,1,2016-05-02
2,2,2017-06-25
3,3,2016-03-02
4,3,2018-07-03


In [9]:
q('''SELECT x.player_id, x.event_date,
     (SELECT SUM(games_played) FROM activity y
      WHERE x.player_id = y.player_id
      AND x.event_date >= y.event_date) as games_played_so_far  
      FROM activity x''')

,player_id,event_date,games_played_so_far
0,1,2016-03-01,5
1,1,2016-05-02,11
2,2,2017-06-25,1
3,3,2016-03-02,0
4,3,2018-07-03,5


### Write an SQL query that reports the fraction of players that logged in again on the day after the day they first logged in, rounded to 2 decimal places. In other words, you need to count the number of players that logged in for at least two consecutive days starting from their first login date, then divide that number by the total number of players.

In [20]:
q('''SELECT player_id, event_date, 
     RANK() OVER(PARTITION BY player_id ORDER BY event_date) as rank_date,
     LEAD(event_date, 1) OVER(PARTITION BY player_id ORDER BY event_date) as lead_date
     FROM activity
     ''')

,player_id,event_date,rank_date,lead_date
0,1,2016-03-01,1,2016-03-02
1,1,2016-03-02,2,None
2,2,2017-06-25,1,None
3,3,2016-03-02,1,2018-07-03
4,3,2018-07-03,2,None


In [30]:
q('''SELECT player_id
     FROM
     (SELECT player_id, event_date, 
      RANK() OVER(PARTITION BY player_id ORDER BY event_date) as rank_date,
      LEAD(event_date, 1) OVER(PARTITION BY player_id ORDER BY event_date) as lead_date
      FROM activity)
     WHERE Date(lead_date, '-1 day') = event_date
''')

,player_id
0,1


In [69]:
q('''SELECT COUNT(player_id)
     FROM
     (SELECT player_id, event_date, 
      RANK() OVER(PARTITION BY player_id ORDER BY event_date) as rank_date,
      LEAD(event_date, 1) OVER(PARTITION BY player_id ORDER BY event_date) as lead_date
      FROM activity)
      WHERE Date(lead_date, '-1 day') = event_date
''')

,COUNT(player_id)
0,1


In [58]:
q('''SELECT COUNT(DISTINCT x.player_id) FROM activity x
''')

,COUNT(DISTINCT x.player_id)
0,3


In [78]:
q('''SELECT 
    (SELECT COUNT(player_id)
     FROM
     (SELECT player_id, event_date, 
      RANK() OVER(PARTITION BY player_id ORDER BY event_date) as rank_date,
      LEAD(event_date, 1) OVER(PARTITION BY player_id ORDER BY event_date) as lead_date
      FROM activity)
      WHERE Date(lead_date, '-1 day') = event_date)
      /(SELECT COUNT(DISTINCT player_id) FROM activity) as Fraction     
''').astype(float)

,Fraction
0,0.0
